In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# Importing main dataset, which has been cleaned
dfIndiaSUF = pd.read_csv('Indian_Start_Up_Data_v1.csv', skipinitialspace = True)

In [3]:
# Deleting first incorrect index column
dfIndiaSUF = dfIndiaSUF.iloc[: , 1:]

# Merging main dataset with GDP data

In [4]:
# Importing GDP data file
dfIndiaGDP = pd.read_csv('Indian GDP Data.csv', skipinitialspace = True)

In [5]:
# Removing columns outside range of 2015-2021, as years are not in main dataset
dfIndiaGDP = dfIndiaGDP[dfIndiaGDP['Year'] > 2014]
dfIndiaGDP = dfIndiaGDP[dfIndiaGDP['Year'] < 2022]

In [6]:
dfIndiaSUF.head(2)

,Startup Name,Founder(s),Funding Year,Funding Month,Funding Day,Founding Date,Industry/Vertical,Funding Date,Sub-Vertical,City,Investor(s),Investment Amount (USD),Investment Type,Remarks
0,Lenskart,NaN,2015,1,2,NaN,Other,02/01/2015,NaN,NaN,"Tpg Growth, Tr Capital, Idg Ventures",2150000,Other,NaN
1,Violetstreet,NaN,2015,1,2,NaN,Other,02/01/2015,NaN,NaN,"Venkat Vallabhneni, Sudhakar Reddy, Srinivasa ...",315000,Early stage,NaN


In [7]:
# Data preview
dfIndiaGDP.head(2)

,Year,GDP_In_Billion_USD,Per_Capita_in_USD,Percentage_Growth
0,2021,3173.40,2277,8.95
1,2020,2667.69,1933,-6.60


In [8]:
# Renaming index columns, to match main data set
dfIndiaGDP = dfIndiaGDP.rename(columns={'Year':'Funding Year', 'Percentage_Growth ':'Yearly GDP Percentage Growth'})

In [9]:
# Left joining the two datasets on 'Founding Year' on main dataset
dfIndiaSUF = pd.merge(dfIndiaSUF, dfIndiaGDP[['Funding Year', 'Yearly GDP Percentage Growth']], how='left', on='Funding Year')

# Merging main dataset with Unicorn data

In [10]:
# Importing Unicorn data file
dfIndiaU = pd.read_csv('Indian Unicorn Data.csv', skipinitialspace = True)

In [11]:
# Deleting first incorrect index column
dfIndiaU = dfIndiaU.iloc[: , 1:]

In [12]:
# Data preview
dfIndiaU.head(2)

,Company,Sector,Entry Valuation^^ ($B),Valuation ($B),Entry,Location,Select Investors
0,InMobi,Adtech - Mobile Ads,1.0,1.0,Sep/2011,Bangalore/Singapore,"KPCB, Sherpalo Ventures, SoftBank"
1,Flipkart^,E-Commerce,1.0,37.6,Feb/2012,Bangalore/Singapore,"Accel, Tiger Global, Naspers, SoftBank, Tencent"


In [13]:
# Renaming index columns, to match main data set
dfIndiaU = dfIndiaU.rename(columns={'Company':'Startup Name', 'Valuation ($B)':'Valuation (B USD)', 'Entry': 'Funding Date*'})

In [14]:
# Adding Unicorn column
dfIndiaU.insert(0, 'Unicorn', 'Yes')

In [15]:
# Left joining the two datasets on 'Startup Name' on main dataset
dfIndiaSUF = pd.merge(dfIndiaSUF, dfIndiaU[['Startup Name', 'Unicorn', 'Valuation (B USD)', 'Funding Date*']], how='left', on='Startup Name')


In [18]:
# Chaning NaN value to 'No' in 'Unicorn' column
dfIndiaSUF.loc[dfIndiaSUF['Unicorn'] != 'Yes', 'Unicorn'] = np.nan

## Cleaning name gender dataset

In [19]:
# Importing Name gender data file
dfIndiaG = pd.read_csv('Indian Name Gender Data.csv', skipinitialspace = True, sep=';')

In [20]:
# Data preview
dfIndiaG.head(2)

,Name,Target
0,Yash,1
1,Prit,1


In [21]:
# Renaming index columns, to match other data set
dfIndiaG = dfIndiaG.rename(columns={'Name':'Founder First Name', 'Target':'Gender ID'})

In [22]:
# Adding gender column based on gender ID value
dfIndiaG.insert(2, 'Founder Gender', np.where(dfIndiaG['Gender ID'] == 1, 'Male', 'Female'))

## Cleaning additional start-up dataset

In [23]:
# Importing additional start-up data file
dfIndiaASUF = pd.read_csv('Indian Start-Up Additional Data.csv', skipinitialspace = True, sep=";")

In [24]:
# Data preview
dfIndiaASUF.head(2)

,Company/Brand,Founded,Headquarters,Sector,What it does,Founder/s,Investor/s,Amount,Stage,Date
0,CollegeDekho,2015.0,Gurgaon,E-learning,"Collegedekho.com is Student‚Äôs Partner, Frien...",Ruchir Arora,"Disrupt ADQ, QIC","$35,000,000",Series B,Dec/21
1,BOX8,2012.0,Mumbai,Food & Beverages,India's Largest Desi Meals Brand,"Anshul Gupta, Amit Raj",Tiger Global,"$40,000,000",NaN,Dec/21


In [25]:
# Renaming index columns, to match other data set
dfIndiaASUF = dfIndiaASUF.rename(columns={'Company/Brand':'Startup Name', 'Founder/s':'Founder Name'})

In [27]:
# Creating a new column from 'Founder Name' with only the first name
dfIndiaASUF.insert(10, 'Founder First Name', dfIndiaASUF['Founder Name'].str.split(" ").str[0])

In [28]:
dfIndiaASUF['Founder First Name'] = dfIndiaASUF['Founder First Name'].replace(',','',regex = True)

# Merging additional start-up dataset with name gender data

In [29]:
# Left joining the two datasets on 'Founding First Name' on additional start-up dataset
dfIndiaASUF = pd.merge(dfIndiaASUF, dfIndiaG[['Founder First Name', 'Founder Gender']], how='left', on='Founder First Name')

In [30]:
#### (418 are unique) 546 of 1366 rows are empty
dfIndiaASUF['Founder Gender'].isnull().sum()

546

In [31]:
# Deleting 'Founder First Name' column
dfIndiaASUF = dfIndiaASUF.drop(['Founder First Name'], axis=1)

In [32]:
dfIndiaASUF.head(2)

,Startup Name,Founded,Headquarters,Sector,What it does,Founder Name,Investor/s,Amount,Stage,Date,Founder Gender
0,CollegeDekho,2015.0,Gurgaon,E-learning,"Collegedekho.com is Student‚Äôs Partner, Frien...",Ruchir Arora,"Disrupt ADQ, QIC","$35,000,000",Series B,Dec/21,NaN
1,BOX8,2012.0,Mumbai,Food & Beverages,India's Largest Desi Meals Brand,"Anshul Gupta, Amit Raj",Tiger Global,"$40,000,000",NaN,Dec/21,Male


# Merging main dataset with additional start-up and name gender dataset

In [33]:
dfIndiaASUF.head(2)

,Startup Name,Founded,Headquarters,Sector,What it does,Founder Name,Investor/s,Amount,Stage,Date,Founder Gender
0,CollegeDekho,2015.0,Gurgaon,E-learning,"Collegedekho.com is Student‚Äôs Partner, Frien...",Ruchir Arora,"Disrupt ADQ, QIC","$35,000,000",Series B,Dec/21,NaN
1,BOX8,2012.0,Mumbai,Food & Beverages,India's Largest Desi Meals Brand,"Anshul Gupta, Amit Raj",Tiger Global,"$40,000,000",NaN,Dec/21,Male


In [34]:
# Corrected one incorrect column
dfIndiaASUF[dfIndiaASUF['Founded'].isnull() == True]
dfIndiaASUF['Founded'] = dfIndiaASUF['Founded'].fillna(2021)

In [35]:
# Changing column to integer data value
dfIndiaASUF['Founded'] = dfIndiaASUF['Founded'].astype(int)

In [36]:
# Renaming index columns, to match main data set
dfIndiaASUF = dfIndiaASUF.rename(columns={'Founded':'Founded Date**', 'Founder/s':'Founder Name'})

In [37]:
dfIndiaASUF_FounderGender = dfIndiaASUF.pop('Founder Gender')
dfIndiaASUF.insert(6, 'Founder Gender', dfIndiaASUF_FounderGender)

In [38]:
# Deleting row duplicates based on several columns
dfIndiaASUF = dfIndiaASUF.drop_duplicates(subset=['Startup Name'])

In [41]:
# Left joining additional start-up dataset on 'Startup Name' on main dataset
dfIndiaSUF = pd.merge(dfIndiaSUF, dfIndiaASUF[['Startup Name', 'Founder Name', 'Founder Gender', 'Founded Date**']], how='left', on='Startup Name')

### Final data cleaning

In [42]:
# Deleting specific date columns
dfIndiaSUF = dfIndiaSUF.drop(['Funding Year', 'Funding Month', 'Funding Day'], axis=1)

In [43]:
# Moving columns, by assigning it, deleting it, and then adding it again
dfIndiaSUF_FounderName = dfIndiaSUF.pop('Founder Name')
dfIndiaSUF.insert(3, 'Founder Name', dfIndiaSUF_FounderName)

dfIndiaSUF_FounderGender = dfIndiaSUF.pop('Founder Gender')
dfIndiaSUF.insert(3, 'Founder Gender', dfIndiaSUF_FounderGender)

dfIndiaSUF_Founders = dfIndiaSUF.pop('Founder(s)')
dfIndiaSUF.insert(2, 'Founder(s)', dfIndiaSUF_Founders)

dfIndiaSUF_FundingDate = dfIndiaSUF.pop('Funding Date')
dfIndiaSUF.insert(9, 'Funding Date', dfIndiaSUF_FundingDate)

dfIndiaSUF_InvestmentType = dfIndiaSUF.pop('Investment Type')
dfIndiaSUF.insert(10, 'Investment Type', dfIndiaSUF_InvestmentType)

In [44]:
#### If 'Founding Date' is empty, then it replaces the value with 'Founding Date**' column with relevant data
dfIndiaSUF['Founding Date'] = np.where(dfIndiaSUF['Founding Date'].isna() == True, dfIndiaSUF['Founded Date**'], dfIndiaSUF['Founding Date'])

In [45]:
# Deleting columns
dfIndiaSUF = dfIndiaSUF.drop(['Remarks', 'Funding Date*', 'Founded Date**', 'Sub-Vertical', 'Founder Name'], axis=1)

In [46]:
# Renaming column
dfIndiaSUF = dfIndiaSUF.rename(columns={'Yearly GDP Percentage Growth': 'Yearly GDP (% Growth)', 'Unicorn': 'Unicorn Company'})

### Updated data set

In [57]:
dfIndiaSUF.head(2)

,Startup Name,Founding Date,Founder(s),Founder Gender,Industry/Vertical,City,Investor(s),Funding Date,Investment Type,Investment Amount (USD),Yearly GDP (% Growth),Unicorn Company,Valuation (B USD)
0,Lenskart,2008.0,NaN,NaN,Other,NaN,"Tpg Growth, Tr Capital, Idg Ventures",02/01/2015,Other,2150000,8.0,Yes,4.5
1,Violetstreet,NaN,NaN,NaN,Other,NaN,"Venkat Vallabhneni, Sudhakar Reddy, Srinivasa ...",02/01/2015,Early stage,315000,8.0,NaN,NaN


In [49]:
# Exporting dataframe data set as csv file
dfIndiaSUF.to_csv('Indian_Start_Up_Data_v2.csv')

In [50]:
# Exporting dataframe data set as excel file
dfIndiaSUF.to_excel('Indian_Start_Up_Data_v2.xlsx')  

In [51]:
dfIndiaSUF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4697 entries, 0 to 4696
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Startup Name             4697 non-null   object 
 1   Founding Date            1947 non-null   float64
 2   Founder(s)               1898 non-null   object 
 3   Founder Gender           542 non-null    object 
 4   Industry/Vertical        4697 non-null   object 
 5   City                     4091 non-null   object 
 6   Investor(s)              4589 non-null   object 
 7   Funding Date             4697 non-null   object 
 8   Investment Type          3317 non-null   object 
 9   Investment Amount (USD)  4697 non-null   int64  
 10  Yearly GDP (% Growth)    4697 non-null   float64
 11  Unicorn Company          154 non-null    object 
 12  Valuation (B USD)        154 non-null    float64
dtypes: float64(3), int64(1), object(9)
memory usage: 513.7+ KB


In [54]:
dfIndiaSUF.shape

(4697, 13)

In [58]:
dfIndiaSUF.isna().sum()

Startup Name                  0
Founding Date              2750
Founder(s)                 2799
Founder Gender             4155
Industry/Vertical             0
City                        606
Investor(s)                 108
Funding Date                  0
Investment Type            1380
Investment Amount (USD)       0
Yearly GDP (% Growth)         0
Unicorn Company            4543
Valuation (B USD)          4543
dtype: int64